In [78]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit

In [14]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Carnê

In [15]:
carne = sc.textFile('CARNE.txt')

In [16]:
header = carne.first()
header

'DATA_CONTRATO\tCODIGO_CONTRATO\tCODIGO_CLIENTE\tDATA_VENCIMENTO\tDATA_PAGAMENTO\tVALOR_PARCELA\tVALOR_ENCARGO\tVALOR_VAGO\tNUMERO_PARCELA\tQTD_PAGAMENTOS\tMODALIDADE\tTIPO_PRODUTO'

In [17]:
#filter out the header, make sure the rest looks correct
carne = carne.filter(lambda line: line != header)

In [18]:
#here's where the changes take place
#this creates a dataframe using whatever pyspark feels like using (I think string is the default). the header.split is providing the names of the columns
carne_df = carne.map(lambda k: k.split("\t")).toDF(header.split("\t"))
carne_df.show()

+--------------------+---------------+--------------+--------------------+--------------------+-------------+-------------+----------+--------------+--------------+----------+------------+
|       DATA_CONTRATO|CODIGO_CONTRATO|CODIGO_CLIENTE|     DATA_VENCIMENTO|      DATA_PAGAMENTO|VALOR_PARCELA|VALOR_ENCARGO|VALOR_VAGO|NUMERO_PARCELA|QTD_PAGAMENTOS|MODALIDADE|TIPO_PRODUTO|
+--------------------+---------------+--------------+--------------------+--------------------+-------------+-------------+----------+--------------+--------------+----------+------------+
|2018/04/04 00:00:...|   100113272222|            10|2018/05/10 00:00:...|2018/05/07 00:00:...|         78.6|        73.89|      78.6|             1|             1|         5|           1|
|2018/04/04 00:00:...|   100113272222|            10|2018/06/10 00:00:...|2018/06/08 00:00:...|         78.6|        70.04|      78.6|             2|             1|         5|           1|
|2017/02/03 00:00:...|   100081371671|            10|20

In [20]:
carne_df_ts = carne_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
carne_df_ts = carne_df_ts.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))
carne_df_ts = carne_df_ts.withColumn("DATA_CONTRATO", to_timestamp("DATA_CONTRATO", "yyyy/MM/dd HH:mm:ss"))

In [21]:
carne_df_ts.show(5)

+-------------------+---------------+--------------+-------------------+-------------------+-------------+-------------+----------+--------------+--------------+----------+------------+
|      DATA_CONTRATO|CODIGO_CONTRATO|CODIGO_CLIENTE|    DATA_VENCIMENTO|     DATA_PAGAMENTO|VALOR_PARCELA|VALOR_ENCARGO|VALOR_VAGO|NUMERO_PARCELA|QTD_PAGAMENTOS|MODALIDADE|TIPO_PRODUTO|
+-------------------+---------------+--------------+-------------------+-------------------+-------------+-------------+----------+--------------+--------------+----------+------------+
|2018-04-04 00:00:00|   100113272222|            10|2018-05-10 00:00:00|2018-05-07 00:00:00|         78.6|        73.89|      78.6|             1|             1|         5|           1|
|2018-04-04 00:00:00|   100113272222|            10|2018-06-10 00:00:00|2018-06-08 00:00:00|         78.6|        70.04|      78.6|             2|             1|         5|           1|
|2017-02-03 00:00:00|   100081371671|            10|2017-03-05 00:00:0

# PAGOU ANTES DO VENCIMENTO

In [51]:
possible_good = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['CODIGO_CONTRATO'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where(carne_df_ts['DATA_PAGAMENTO'] < carne_df_ts['DATA_VENCIMENTO'])

# PAGOU DEPOIS DO VENCIMENTO

In [79]:
bad_carne = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['CODIGO_CONTRATO'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where(carne_df_ts['DATA_PAGAMENTO'] > carne_df_ts['DATA_VENCIMENTO'])

In [80]:
bad_carne.show(5)

+--------------+---------------+-------------------+-------------------+
|CODIGO_CLIENTE|CODIGO_CONTRATO|     DATA_PAGAMENTO|    DATA_VENCIMENTO|
+--------------+---------------+-------------------+-------------------+
|            10|   100089793914|2017-06-14 00:00:00|2017-06-10 00:00:00|
|            10|   100089793914|2017-07-11 00:00:00|2017-07-10 00:00:00|
|            10|   100089793914|2017-09-08 00:00:00|2017-08-10 00:00:00|
|            15|   100112867545|2018-05-21 00:00:00|2018-05-20 00:00:00|
|            15|   100112867545|2018-06-21 00:00:00|2018-06-20 00:00:00|
+--------------+---------------+-------------------+-------------------+
only showing top 5 rows



In [81]:
bad_client_carne = bad_carne.select("CODIGO_CLIENTE").dropDuplicates()

In [82]:
bad_client_carne.cache()

DataFrame[CODIGO_CLIENTE: string]

In [83]:
bad_client_carne = bad_client_carne.withColumn("CLASS", lit(0))
bad_client_carne.show()

+--------------+-----+
|CODIGO_CLIENTE|CLASS|
+--------------+-----+
|          1090|    0|
|          1512|    0|
|          2069|    0|
|          5325|    0|
|          9586|    0|
|         11888|    0|
|         13772|    0|
|         17401|    0|
|         17686|    0|
|         20428|    0|
|         21248|    0|
|         21452|    0|
|         27317|    0|
|         28839|    0|
|         29454|    0|
|         30002|    0|
|         32558|    0|
|         32812|    0|
|         33174|    0|
|         38986|    0|
+--------------+-----+
only showing top 20 rows



# Cadastro

In [23]:
cadastral = sc.textFile('CADASTRAL.txt')

In [24]:
header1 = cadastral.first()
header1

'CODIGO_CLIENTE\tDATA_NASCIMENTO\tSEXO\tESTADO_CIVIL\tCIDADE\tUF\tBAIRRO\tCEP\tQTD_DEPENDENTES\tDATA_CADASTRO\tDATA_ADMISSAO\tCATEGORIAL_PROFISSAO\tTIPO_RESIDENCIA\tRENDA_TITULAR\tRENDA_CJ\tOUTRAS_RENDAS\tQTD_CARTOES_ADICIONAIS\tFLAG_CONTA_BANCO\tVALOR_HISTORICO_COMPRAS'

In [25]:
cadastral = cadastral.filter(lambda line: line != header1)

In [26]:
cadastral_df = cadastral.map(lambda k: k.split("\t")).toDF(header1.split("\t"))
cadastral_df.show(5)

+--------------+--------------------+----+------------+-------------+---+--------+--------+---------------+--------------------+--------------------+--------------------+---------------+-------------+--------+-------------+----------------------+----------------+-----------------------+
|CODIGO_CLIENTE|     DATA_NASCIMENTO|SEXO|ESTADO_CIVIL|       CIDADE| UF|  BAIRRO|     CEP|QTD_DEPENDENTES|       DATA_CADASTRO|       DATA_ADMISSAO|CATEGORIAL_PROFISSAO|TIPO_RESIDENCIA|RENDA_TITULAR|RENDA_CJ|OUTRAS_RENDAS|QTD_CARTOES_ADICIONAIS|FLAG_CONTA_BANCO|VALOR_HISTORICO_COMPRAS|
+--------------+--------------------+----+------------+-------------+---+--------+--------+---------------+--------------------+--------------------+--------------------+---------------+-------------+--------+-------------+----------------------+----------------+-----------------------+
|             3|1951/10/21 00:00:...|   F|          CA|   SANTA ROSA| RS|  GLORIA|98900000|              1|1998/10/28 00:00:...|1990/01/

# Contrato

In [27]:
contrato = sc.textFile('CONTRATO.txt')

In [28]:
header2 = contrato.first()
header2

'CODIGO_CLIENTE\tCODIGO_CONTRATO\tVALOR_CONTRATO\tQTD_PARCELAS\tDATA_OPERACAO\tFLAG_TITULAR'

In [29]:
contrato = contrato.filter(lambda line: line != header2)

In [30]:
contrato_df = contrato.map(lambda k: k.split("\t")).toDF(header2.split("\t"))
contrato_df.show(5)

+--------------+---------------+--------------+------------+--------------------+------------+
|CODIGO_CLIENTE|CODIGO_CONTRATO|VALOR_CONTRATO|QTD_PARCELAS|       DATA_OPERACAO|FLAG_TITULAR|
+--------------+---------------+--------------+------------+--------------------+------------+
|            10|   100081371671|        1528.0|           1|2017/02/03 00:00:...|           S|
|            10|   100089793914|         362.7|           6|2017/05/23 00:00:...|           S|
|            10|   100113272222|        392.98|           5|2018/04/04 00:00:...|           S|
|            15|   100087239845|       1133.48|           2|2017/04/18 00:00:...|            |
|            15|   100098395860|       3078.55|          10|2017/09/18 00:00:...|           S|
+--------------+---------------+--------------+------------+--------------------+------------+
only showing top 5 rows



# Cartão

In [31]:
cartao = sc.textFile('CARTAO.txt')

In [32]:
header3 = cartao.first()
header3

'CODIGO_CLIENTE\tCODIGO_CARTAO\tVALOR_MORA\tVALOR_MULTA\tVALOR_TOTAL\tVALOR_MIN\tDATA_VENCIMENTO\tVALOR_PAGO\tTARIFA\tOUTROS_VALORES\tLIMITE\tSOMA_VALOR_DIVIDAS_FUTURAS\tDATA_PAGAMENTO\tQTD_CONTRATOS'

In [33]:
cartao = cartao.filter(lambda line: line != header3)

In [35]:
cartao_df = cartao.map(lambda k: k.split("\t")).toDF(header3.split("\t"))
cartao_df.show(5)

+--------------+-------------+----------+-----------+-----------+---------+--------------------+----------+------+--------------+------+--------------------------+--------------------+-------------+
|CODIGO_CLIENTE|CODIGO_CARTAO|VALOR_MORA|VALOR_MULTA|VALOR_TOTAL|VALOR_MIN|     DATA_VENCIMENTO|VALOR_PAGO|TARIFA|OUTROS_VALORES|LIMITE|SOMA_VALOR_DIVIDAS_FUTURAS|      DATA_PAGAMENTO|QTD_CONTRATOS|
+--------------+-------------+----------+-----------+-----------+---------+--------------------+----------+------+--------------+------+--------------------------+--------------------+-------------+
|            23|    105028440|       0.0|        0.0|     298.79|    64.55|2016/06/10 00:00:...|    298.79|   0.0|          5.99|2604.0|                    8118.6|2016/06/09 00:00:...|            5|
|            23|    105670225|       0.0|        0.0|     307.34|    66.26|2016/07/10 00:00:...|    307.34|   0.0|          5.99|2604.0|                   7811.26|2016/07/08 00:00:...|            6|
|    

In [84]:
cartao_df = cartao_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
cartao_df = cartao_df.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))

In [85]:
cartao_df.show(5)

+--------------+-------------+----------+-----------+-----------+---------+-------------------+----------+------+--------------+------+--------------------------+-------------------+-------------+
|CODIGO_CLIENTE|CODIGO_CARTAO|VALOR_MORA|VALOR_MULTA|VALOR_TOTAL|VALOR_MIN|    DATA_VENCIMENTO|VALOR_PAGO|TARIFA|OUTROS_VALORES|LIMITE|SOMA_VALOR_DIVIDAS_FUTURAS|     DATA_PAGAMENTO|QTD_CONTRATOS|
+--------------+-------------+----------+-----------+-----------+---------+-------------------+----------+------+--------------+------+--------------------------+-------------------+-------------+
|            23|    105028440|       0.0|        0.0|     298.79|    64.55|2016-06-10 00:00:00|    298.79|   0.0|          5.99|2604.0|                    8118.6|2016-06-09 00:00:00|            5|
|            23|    105670225|       0.0|        0.0|     307.34|    66.26|2016-07-10 00:00:00|    307.34|   0.0|          5.99|2604.0|                   7811.26|2016-07-08 00:00:00|            6|
|            23

# PAGAMENTO ANTES DO VENCIMENTO

In [97]:
good_cartao = cartao_df.select(cartao_df['CODIGO_CLIENTE'],cartao_df['DATA_PAGAMENTO'],cartao_df['DATA_VENCIMENTO']).where(cartao_df['DATA_PAGAMENTO'] < cartao_df['DATA_VENCIMENTO'])

In [98]:
good_cartao.show(5)

+--------------+-------------------+-------------------+
|CODIGO_CLIENTE|     DATA_PAGAMENTO|    DATA_VENCIMENTO|
+--------------+-------------------+-------------------+
|            23|2016-06-09 00:00:00|2016-06-10 00:00:00|
|            23|2016-07-08 00:00:00|2016-07-10 00:00:00|
|            23|2016-08-08 00:00:00|2016-08-10 00:00:00|
|            23|2016-09-06 00:00:00|2016-09-10 00:00:00|
|            23|2016-10-07 00:00:00|2016-10-10 00:00:00|
+--------------+-------------------+-------------------+
only showing top 5 rows



In [99]:
possible_good = good_cartao.select('CODIGO_CLIENTE').dropDuplicates()

In [109]:
possible_good.count()

583309

In [110]:
"""test = possible_good.select('CODIGO_CLIENTE').where(possible_good['CODIGO_CLIENTE'].isin(bad_client_cartao['CODIGO_CLIENTE']) == True)
test.count()"""

"test = possible_good.select('CODIGO_CLIENTE').where(possible_good['CODIGO_CLIENTE'].isin(bad_client_cartao['CODIGO_CLIENTE']) == True)\ntest.count()"

# PAGAMENTO DEPOIS DO VENCIMENTO

In [87]:
bad_cartao = cartao_df.select(cartao_df['CODIGO_CLIENTE'],cartao_df['DATA_PAGAMENTO'],cartao_df['DATA_VENCIMENTO']).where(cartao_df['DATA_PAGAMENTO'] > cartao_df['DATA_VENCIMENTO'])

In [89]:
bad_cartao.show(5)

+--------------+-------------------+-------------------+
|CODIGO_CLIENTE|     DATA_PAGAMENTO|    DATA_VENCIMENTO|
+--------------+-------------------+-------------------+
|            28|2016-07-11 00:00:00|2016-07-10 00:00:00|
|            28|2016-08-11 00:00:00|2016-08-10 00:00:00|
|            28|2016-09-12 00:00:00|2016-09-10 00:00:00|
|            28|2016-10-14 00:00:00|2016-10-10 00:00:00|
|            28|2016-11-12 00:00:00|2016-11-10 00:00:00|
+--------------+-------------------+-------------------+
only showing top 5 rows



In [90]:
bad_client_cartao = bad_cartao.select('CODIGO_CLIENTE').dropDuplicates()

In [92]:
bad_client_cartao.cache()

DataFrame[CODIGO_CLIENTE: string]

In [91]:
bad_client_cartao.show(10)

+--------------+
|CODIGO_CLIENTE|
+--------------+
|          1090|
|          2069|
|          9586|
|         11888|
|         17401|
|         17686|
|         20428|
|         21248|
|         21452|
|         29865|
+--------------+
only showing top 10 rows



In [95]:
bad_client_cartao = bad_client_cartao.withColumn("CLASS", lit(0))

In [96]:
bad_client_cartao.show()

+--------------+-----+
|CODIGO_CLIENTE|CLASS|
+--------------+-----+
|          1090|    0|
|          2069|    0|
|          9586|    0|
|         11888|    0|
|         17401|    0|
|         17686|    0|
|         20428|    0|
|         21248|    0|
|         21452|    0|
|         29865|    0|
|         32558|    0|
|         32812|    0|
|         33174|    0|
|         39641|    0|
|         44032|    0|
|         44717|    0|
|         47919|    0|
|         48738|    0|
|         52206|    0|
|         52800|    0|
+--------------+-----+
only showing top 20 rows

